In [27]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import datetime, json
from ipywidgets import HTML
from IPython.display import display
import base64

def make_box_layout():
    """define how the widget box looks"""
    return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )


    

html_save_buttons = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="{filename}" href="data:text/csv;base64,{payload}" download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Save Values</button>
</a>
</body>
</html>
'''

class HubbleFitter(widgets.HBox):
    def __init__(self):
        super().__init__()
        plt.close()
        self.output = widgets.Output() # This is where the plot will go
        with self.output:
            figTitle = "Hubble Diagram"
            self.fig, self.ax = plt.subplots(
                constrained_layout=True, figsize=(5, 3.5), num=figTitle)
            self.ax.set_xlim(0.0, 80.0)
            self.ax.set_ylim(0.0, 7000.0)
            self.xlabel = self.ax.set_xlabel("Distance (Mpc)")
            self.ylabel = self.ax.set_ylabel("Velocity (km/sec)")
            self.title = self.ax.set_title("H0")
            self.fig.canvas.toolbar_position = 'bottom'
            self.ax.grid(True)
    
        # DEFINE WIDGETS

        #FILE
        self.filename = 'HubbleFitter.txt'
        self.enteredDataStr = json.dumps({"nothing yet":0})
        self.b64 = base64.b64encode(self.enteredDataStr.encode())
        self.payload = self.b64.decode()

        #BUTTON to "Save Progress"
        self.html_save_button = widgets.HTML(html_save_buttons.format(payload=self.payload,filename=self.filename))


        self.uploader = widgets.FileUpload(
            accept='.txt',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
            multiple=False,  # True to accept multiple files upload else False,
            layout=widgets.Layout(width='auto', height='40px'),
            button_style='success'    
        )
        self.uploader.description="Restore Values from File"
        self.uploader.observe(self.parseUpload, names='value')

        columnNames = ["Name","Size [mrad]", "D [Mpc]", "Ca K [A]", "Ca K [z]", "Ca H [A]", "Ca H [z]", "Ha [A]", "Ha [z]", "v [km/s]", "include?"]
        layout = widgets.Layout(width='70px', height='29px')
        headers = [widgets.Label(value=columnName, layout=layout) for columnName in columnNames]
        header = widgets.HBox(headers)
        self.h0Max = widgets.FloatSlider(value=350,min=1,max=500.0,step=0.1, description='H0 Max', disabled=False, continuous_update=False, orientation='horizontal',readout=True,readout_format='.1f')
        self.h0Min = widgets.FloatSlider(value=10,min=1,max=500.0,step=0.1, description='H0 Min', disabled=False, continuous_update=False, orientation='horizontal',readout=True,readout_format='.1f')
        self.wrows = [widgets.HBox([self.html_save_button, self.uploader, widgets.VBox([self.h0Max, self.h0Min])]), header]
        self.rows = []
        self.galaxyList = open("galaxy-list.txt",'r').readlines()
        for galaxy in self.galaxyList:
            row = [
                widgets.Label(value=galaxy, layout=layout), 
                widgets.FloatText(disabled=False, layout=layout), 
                widgets.Label(value="",layout=layout),
                widgets.FloatText(disabled=False, layout=layout), 
                widgets.Label(value="",layout=layout),
                widgets.FloatText(disabled=False, layout=layout), 
                widgets.Label(value="",layout=layout),
                widgets.FloatText(disabled=False, layout=layout), 
                widgets.Label(value="",layout=layout),
                widgets.Label(value="",layout=layout),
                widgets.Checkbox(value=False,description='',disabled=False, indent=False, layout=widgets.Layout(width='29px', height='29px'))
                  ]
            self.rows.append(row)
            self.wrows.append(widgets.HBox(row))

        self.table = widgets.VBox(self.wrows)

        self.plotPoints, = self.ax.plot([-100],[-100],'r*') # Plot one point off scale, to define self.dataPoints
        self.h0MinPoints, = self.ax.plot([-100],[-100],'r:') # Plot one point off scale, to define self.dataPoints
        self.h0MaxPoints, = self.ax.plot([-100],[-100],'r:') # Plot one point off scale, to define self.dataPoints
    
        self.table.layout = make_box_layout()
        self.out_box = widgets.Box([self.output])
        self.output.layout = make_box_layout()

        self.children = [self.table, self.output]
        self._setupObserve()
        
        self._updatePlot()
        
    def on_value_change(self,change):
        enteredData = {}
        enteredData['h0Min'] =  self.h0Min.value
        enteredData['h0Max'] =  self.h0Max.value
        for row in self.rows:
            name = row[0].value
            enteredData[name] = [row[1].value, row[3].value, row[5].value, row[7].value, row[10].value]
            size = row[1].value
            if size > 0.0:
                distance = "%.1f"%(22/size)
            else:
                distance = ""
            row[2].value = distance

            self.update_z(row, 3, 3933.7)
            self.update_z(row, 5, 3968.5)
            self.update_z(row, 7, 6562.8)
        self._updatePlot()
        enteredDataStr = json.dumps(enteredData)
        b64 = base64.b64encode(enteredDataStr.encode())
        payload = b64.decode()
        self.html_save_button.value = html_save_buttons.format(payload=payload,filename=self.filename)

    def _setupObserve(self):
        for row in self.rows:
            for i in [1,3,5,7,10]:
                row[i].observe(self.on_value_change, names='value')
        self.h0Min.observe(self.on_value_change, names='value')
        self.h0Max.observe(self.on_value_change, names='value')
        
                
    def _updatePlot(self):
        xdata = []
        ydata = []
        for iRow,row in enumerate(self.rows):
            if row[10].value:
                try:
                    xx = float(row[2].value)
                    yy = float(row[9].value)
                    xdata.append(xx)
                    ydata.append(yy)
                except ValueError:
                    pass
        if len(xdata) == 0:
            xdata = [-100]
            ydata = [-100]
        
        self.plotPoints.set_xdata(xdata)
        self.plotPoints.set_ydata(ydata)
        self.h0MinPoints.set_xdata([0,100])
        h0Min = float(str(self.h0Min.value))
        self.h0MinPoints.set_ydata([0,100*h0Min])
        self.h0MaxPoints.set_xdata([0,100])
        h0Max = float(str(self.h0Max.value))
        self.h0MaxPoints.set_ydata([0,100*h0Max])
        self.title.set_text("H0 max = %.1f   H0 min = %.1f"%(h0Max, h0Min))
    def parseUpload(self, change):
        key = list(self.uploader.value.keys())[0]
        x = self.uploader.value[key]
        d = json.loads(x['content'])
        print("d.keys()=",d.keys())
        for key in d.keys():
            if "h0" not in key:              
                inputRow = d[key]
                iOutputRow = self.galaxyList.index(key)
                outputRow = self.rows[iOutputRow]
                iOutRows = [1, 3, 5, 7, 10]
                for i in range(5): 
                    temp =  inputRow[i]
                    outputRow[iOutRows[i]].value =  temp
        self.h0Min.value = d['h0Min']
        self.h0Max.value = d['h0Max']

        self._updatePlot()
        
    def update_z(self, row, i, restWavelength):
        w = row[i].value
        if w > 0:
            row[i+1].value = "%.4f"%((w-restWavelength)/restWavelength)
        else:
            row[i+1].value = ""

        zSum = 0
        nz = 0
        for i in [4, 6, 8]:
            try:
                zSum += float(row[i].value)
                nz += 1
            except ValueError:
                break
        if nz == 3:
            velocity = (zSum/3.0)*3e5 # recessional velocity in km/sec
            row[9].value = "%.0f"%velocity

HubbleFitter()


HubbleFitter(children=(VBox(children=(HBox(children=(HTML(value='<html>\n<head>\n<meta name="viewport" content…